In [ ]:
!pip install unidecode

In [ ]:
import pandas as pd
import chardet
import numpy as np
import glob
from unidecode import unidecode
from natsort import natsorted
import time
import requests
from bs4 import BeautifulSoup
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Loading Dataset

In [ ]:
df = pd.read_csv('drive/MyDrive/Colab Notebooks/FYP/Gameweek Data/pre_dataset.csv')

<ipython-input-66-400fcc5e0083>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('drive/MyDrive/Colab Notebooks/FYP/Gameweek Data/pre_dataset.csv')


### Merging DFs of all seasons

In [ ]:
df = pd.read_csv('drive/MyDrive/Colab Notebooks/FYP/Gameweek Data/cleaned_merged_seasons.csv')

# Dropping 2016-17 because of unavailable xG and xGA data

df.drop(df.index[df['season_x'] == '2016-17'], inplace=True)

# Dropping 2017-18 because of missing data and replacing it with 2017-18 file

df.drop(df.index[df['season_x'] == '2017-18'], inplace=True)

with open('drive/MyDrive/Colab Notebooks/FYP/Gameweek Data/merged_gw_2017-18.csv', 'rb') as f:
  result = chardet.detect(f.read())

merged_2017_18 = pd.read_csv('drive/MyDrive/Colab Notebooks/FYP/Gameweek Data/merged_gw_2017-18.csv', encoding=result['encoding'])
merged_2017_18['season_x'] = '2017-18'

df = pd.concat([df, merged_2017_18])[df.columns]

# Adding 2018-19

with open('drive/MyDrive/Colab Notebooks/FYP/Gameweek Data/merged_gw_2018-19.csv', 'rb') as f:
  result = chardet.detect(f.read())

merged_2018_19 = pd.read_csv('drive/MyDrive/Colab Notebooks/FYP/Gameweek Data/merged_gw_2018-19.csv', encoding=result['encoding'])
merged_2018_19['season_x'] = '2018-19'

df = pd.concat([df, merged_2018_19])[df.columns]

# Adding 2019-20

with open('drive/MyDrive/Colab Notebooks/FYP/Gameweek Data/merged_gw_2019-20.csv', 'rb') as f:
  result = chardet.detect(f.read())

merged_2019_20 = pd.read_csv('drive/MyDrive/Colab Notebooks/FYP/Gameweek Data/merged_gw_2019-20.csv', encoding=result['encoding'])
merged_2019_20['season_x'] = '2019-20'

df = pd.concat([df, merged_2019_20])[df.columns]

# Adding Current Season Data

with open('drive/MyDrive/Colab Notebooks/FYP/Gameweek Data/merged_gw_current_season.csv', 'rb') as f:
  result = chardet.detect(f.read())

merged_2023_24 = pd.read_csv('drive/MyDrive/Colab Notebooks/FYP/Gameweek Data/merged_gw_current_season.csv', encoding=result['encoding'])
merged_2023_24['season_x'] = '2023-24'

df = pd.concat([df, merged_2023_24])[df.columns]

# Matching name

df['name'] = df['name'].str.replace('_', ' ').str.replace('\d', '', regex=True)
df['name'] = np.where((df['season_x'] == '2018-19') | (df['season_x'] == '2019-20'), df['name'].str.slice(stop=-1), df['name'])

# Removing redundant columns

df = df.drop(columns=['assists', 'fixture', 'goals_conceded', 'goals_scored', 'penalties_missed', 'penalties_saved', 'bonus', 'selected', 'transfers_balance', 'transfers_in', 'transfers_out', 'value', 'opp_team_name', 'own_goals', 'round', 'team_a_score', 'team_h_score', 'team'])
df = df.rename(columns={'total_points': 'points'})

# Fixing Incorrect GW Number for 2019-20

mapping = {39: 30, 40: 31, 41: 32, 42: 33, 43: 34, 44: 35, 45: 36, 46: 37, 47: 38}
df['GW'] = df['GW'].replace(mapping)

<ipython-input-3-b7a22f5efcba>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('drive/MyDrive/Colab Notebooks/FYP/Gameweek Data/cleaned_merged_seasons.csv')


### Updating Current Season Data

In [ ]:
url = "https://fantasy.premierleague.com/api/bootstrap-static/"

response = requests.get(url)

if response.status_code == 200:
  data = response.json()
  players = data['elements']
  players_sorted = sorted(players, key=lambda x: x['id'])

else:
  print(f"Failed to retrieve data. Status code: {response.status_code}")

# Checking if Dataset is up to date

max_gw = 0

for player in players_sorted:
    url = 'https://fantasy.premierleague.com/api/element-summary/' + str(player['id']) + '/'

    response = requests.get(url)

    if response.status_code == 200:
      data = response.json()

    gw = data['history'][-1]['round']

    if gw > max_gw:
      max_gw = gw

if df.iloc[-1]['season_x'] == '2023-24' and df.iloc[-1]['GW'] == max_gw:
  print('Dataset is up to date')

# If not up to date, update

else:
  for player in players_sorted:
    url = 'https://fantasy.premierleague.com/api/element-summary/' + str(player['id']) + '/'

    response = requests.get(url)

    if response.status_code == 200:
      data = response.json()

      if data['history'][-1]['round'] == max_gw:

        season_x = '2023-24'

        name = player['first_name'] + ' ' + player['second_name']

        position = player['element_type']

        if position == 1:
          position = 'GKP'
        elif position == 2:
          position = 'DEF'
        elif position == 3:
          position = 'MID'
        elif position == 4:
          position = 'FWD'

        team = player['team']

        element = player['id']

        bps = data['history'][-1]['bps']

        kickoff_time = data['history'][-1]['kickoff_time']

        opp_team = data['history'][-1]['opponent_team']

        is_home = data['history'][-1]['was_home']

        gw = data['history'][-1]['round']

        influence = data['history'][-1]['influence']

        creativity = data['history'][-1]['creativity']

        threat = data['history'][-1]['threat']

        ict_index = data['history'][-1]['ict_index']

        minutes =  data['history'][-1]['minutes']

        total_points = data['history'][-1]['total_points']

        clean_sheets = data['history'][-1]['clean_sheets']

        saves = data['history'][-1]['saves']

        yellow_cards = data['history'][-1]['yellow_cards']

        red_cards = data['history'][-1]['red_cards']

        new_record = pd.DataFrame([{'season_x': season_x, 'name': name, 'position': position, 'element': element, 'bps': bps, 'kickoff_time': kickoff_time, 'opponent_team': opp_team, 'was_home': is_home, 'GW': gw, 'influence': influence, 'creativity': creativity, 'threat': threat, 'ict_index': ict_index, 'minutes': minutes, 'points': total_points, 'clean_sheets': clean_sheets, 'saves': saves, 'yellow_cards': yellow_cards, 'red_cards': red_cards}])

        df = pd.concat([df, new_record], ignore_index=True)

      # If Double Gameweek:

      if len(data['history']) > 1 and data['history'][-2]['round'] == max_gw:

        bps = data['history'][-2]['bps']

        kickoff_time = data['history'][-2]['kickoff_time']

        opp_team = data['history'][-2]['opponent_team']

        is_home = data['history'][-2]['was_home']

        influence = data['history'][-2]['influence']

        creativity = data['history'][-2]['creativity']

        threat = data['history'][-2]['threat']

        ict_index = data['history'][-2]['ict_index']

        minutes =  data['history'][-2]['minutes']

        total_points = data['history'][-2]['total_points']

        clean_sheets = data['history'][-2]['clean_sheets']

        saves = data['history'][-2]['saves']

        yellow_cards = data['history'][-2]['yellow_cards']

        red_cards = data['history'][-2]['red_cards']

        new_record = pd.DataFrame([{'season_x': season_x, 'name': name, 'position': position, 'element': element, 'bps': bps, 'kickoff_time': kickoff_time, 'opponent_team': opp_team, 'was_home': is_home, 'GW': gw, 'influence': influence, 'creativity': creativity, 'threat': threat, 'ict_index': ict_index, 'minutes': minutes, 'points': total_points, 'clean_sheets': clean_sheets, 'saves': saves, 'yellow_cards': yellow_cards, 'red_cards': red_cards}])

        df = pd.concat([df, new_record], ignore_index=True)

    else:
      print(f"Failed to retrieve data. Status code: {response.status_code}")

  df.to_csv('pre_dataset.csv', index=False)
  files.download('pre_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Fetching Next Gameweek Data using FPL API

In [ ]:
min_gw = 100

for player in players_sorted:
    url = 'https://fantasy.premierleague.com/api/element-summary/' + str(player['id']) + '/'

    response = requests.get(url)

    if response.status_code == 200:
      data = response.json()

    gw = data['fixtures'][0]['event']

    if gw < min_gw:
      min_gw = gw

for player in players_sorted:
  url = 'https://fantasy.premierleague.com/api/element-summary/' + str(player['id']) + '/'

  response = requests.get(url)

  if response.status_code == 200:
    data = response.json()

    if data['fixtures'][0]['event'] == min_gw:

      season_x = '2023-24'

      name = player['first_name'] + ' ' + player['second_name']

      position = player['element_type']

      if position == 1:
        position = 'GKP'
      elif position == 2:
        position = 'DEF'
      elif position == 3:
        position = 'MID'
      elif position == 4:
        position = 'FWD'

      team = player['team']

      element = player['id']

      kickoff_time = data['fixtures'][0]['kickoff_time']

      if data['fixtures'][0]['is_home'] == True:
        opp_team = data['fixtures'][0]['team_a']
      else:
        opp_team = data['fixtures'][0]['team_h']

      is_home = data['fixtures'][0]['is_home']

      gw = data['fixtures'][0]['event']

      new_record = pd.DataFrame([{'season_x': season_x, 'name': name, 'position': position, 'element': element, 'kickoff_time': kickoff_time, 'opponent_team': opp_team, 'was_home': is_home, 'GW': gw, 'ict_index': 0, 'minutes': 0, 'total_points': 0, 'bps': 0}])

      df = pd.concat([df, new_record], ignore_index=True)

    # If Double Gameweek:

    if data['fixtures'][1]['event'] == min_gw:

      kickoff_time = data['fixtures'][1]['kickoff_time']

      if data['fixtures'][1]['is_home'] == True:
        opp_team = data['fixtures'][1]['team_a']
      else:
        opp_team = data['fixtures'][1]['team_h']

      is_home = data['fixtures'][1]['is_home']

      new_record = pd.DataFrame([{'season_x': season_x, 'name': name, 'position': position, 'element': element, 'kickoff_time': kickoff_time, 'opponent_team': opp_team, 'was_home': is_home, 'GW': gw, 'ict_index': 0, 'minutes': 0, 'total_points': 0, 'bps': 0}])

      df = pd.concat([df, new_record], ignore_index=True)

  else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

### Replacing Non-English Characters in Player Names

In [ ]:
df['influence'] = pd.to_numeric(df['influence'], errors='coerce')
df['creativity'] = pd.to_numeric(df['creativity'], errors='coerce')
df['threat'] = pd.to_numeric(df['threat'], errors='coerce')
df['ict_index'] = pd.to_numeric(df['ict_index'], errors='coerce')

In [ ]:
df['name'] = df['name'].apply(unidecode)
df['name'] = df['name'].replace('David De Gea', 'David de Gea')
df['name'] = df['name'].replace('Djordje Petrovic', 'Dorde Petrovic')
df = df.sort_values(['season_x', 'name', 'element', 'GW', 'kickoff_time'])

In [ ]:
df['player_game_x'] = df.groupby(['name', 'season_x']).cumcount() + 1 # determines how many games in the season the player has been at the club for

In [ ]:
df

,season_x,name,position,bps,clean_sheets,creativity,element,ict_index,influence,kickoff_time,minutes,opponent_team,red_cards,saves,threat,points,was_home,yellow_cards,GW,player_game_x
76317,2017-18,Aaron Cresswell,NaN,3,0,0.6,443,1.9,0.4,2017-08-13T15:00:00Z,9,12,0,0,18.0,0,False,0,1,1
82561,2017-18,Aaron Cresswell,NaN,16,0,52.9,443,7.4,21.2,2017-08-19T14:00:00Z,90,14,0,0,0.0,1,False,0,2,2
89169,2017-18,Aaron Cresswell,NaN,11,0,1.6,443,2.4,11.2,2017-08-26T14:00:00Z,90,13,0,0,11.0,1,False,0,3,3
95474,2017-18,Aaron Cresswell,NaN,23,1,47.1,443,6.2,14.2,2017-09-11T19:00:00Z,90,8,0,0,1.0,6,True,0,4,4
96018,2017-18,Aaron Cresswell,NaN,30,1,34.3,443,4.6,11.2,2017-09-16T14:00:00Z,90,19,0,0,0.0,9,False,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168551,2023-24,Zeki Amdouni,FWD,3,0,1.3,594,0.2,0.2,2024-04-20T14:00:00Z,5,17,0,0,0.0,1,False,0,34,34
169572,2023-24,Zeki Amdouni,FWD,28,0,0.3,594,6.6,40.0,2024-04-27T14:00:00Z,33,14,0,0,26.0,7,False,0,35,35
170477,2023-24,Zeki Amdouni,FWD,2,0,1.3,594,0.3,0.0,2024-05-04T14:00:00Z,5,15,0,0,2.0,1,True,0,36,36
171485,2023-24,Zeki Amdouni,FWD,3,0,0.0,594,0.0,0.0,2024-05-11T14:00:00Z,2,18,0,0,0.0,1,False,0,37,37


## Adding Missing Positions

### Changing GK to GKP

In [ ]:
df['position'] = df['position'].replace('GK', 'GKP')

### 2017-18

In [ ]:
# Adding Web Names (df uses full names while positions uses web names)

players_raw_2017_18 = pd.read_csv('drive/MyDrive/Colab Notebooks/FYP/Players Raw Data/players_raw_2017-18.csv')

players_raw_2017_18['name'] = players_raw_2017_18[['first_name', 'second_name']].agg(' '.join, axis=1)

players_raw_2017_18 = players_raw_2017_18.drop_duplicates(subset=['web_name', 'name'], keep='first')

players_raw_2017_18['name'] = players_raw_2017_18['name'].apply(unidecode)

df = df.merge(players_raw_2017_18[['name', 'web_name']], how='left', on='name')

df['web_name'] = df['web_name'].apply(lambda x: unidecode(x) if isinstance(x, str) else x)

# Adding Positions

positions_2017_18 = pd.read_csv('drive/MyDrive/Colab Notebooks/FYP/Position Data/2017-18.csv')

positions_2017_18_unique = positions_2017_18.drop_duplicates(subset='web_name', keep='first') # Removing duplicate web names since not doing so would result in the same player having more rows of data

df = df.merge(positions_2017_18_unique[['web_name', 'position']], how='left', on='web_name')

df['position_x'] = df['position_x'].fillna(df['position_y'])

df = df.drop('position_y', axis=1)
df = df.drop('web_name', axis=1)
df.rename(columns={'position_x': 'position'}, inplace=True)

In [ ]:
# Adding Positions of Players with the same Web Name:

df['position'] = np.where((df['name'] == 'Ashley Barnes') & (df['season_x'] == '2017-18'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Harvey Barnes') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Oliver Burke') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Reece Burke') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Andre Carrillo') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Guido Carrillo') & (df['season_x'] == '2017-18'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Andy Carroll') & (df['season_x'] == '2017-18'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Tom Carroll') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Nathaniel Chalobah') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Trevoh Chalobah') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Joel Coleman') & (df['season_x'] == '2017-18'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Seamus Coleman') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Ben Davies') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Tom Davies') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Ashley Fletcher') & (df['season_x'] == '2017-18'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Darren Fletcher') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Andre Gray') & (df['season_x'] == '2017-18'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Demarai Gray') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Sam Hughes') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Will Hughes') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Ben Johnson') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Glen Johnson') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Curtis Jones') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Phil Jones') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Andy King') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Joshua King') & (df['season_x'] == '2017-18'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Kevin Long') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Shane Long') & (df['season_x'] == '2017-18'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Antonio Martinez Lopez') & (df['season_x'] == '2017-18'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Damian Emiliano Martinez') & (df['season_x'] == '2017-18'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Alex McCarthy') & (df['season_x'] == '2017-18'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'James McCarthy') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Benjamin Mendy') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Nampalys Mendy') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Daryl Murphy') & (df['season_x'] == '2017-18'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Jacob Murphy') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Jamie Murphy') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Alex Palmer') & (df['season_x'] == '2017-18'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Kasey Palmer') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Ayoze Perez') & (df['season_x'] == '2017-18'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Lucas Perez') & (df['season_x'] == '2017-18'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Diafra Sakho') & (df['season_x'] == '2017-18'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Mamadou Sakho') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Alexis Sanchez') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Davinson Sanchez') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Danny Simpson') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Jack Simpson') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Dale Stephens') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Jack Stephens') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Dujon Sterling') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Kazaiah Sterling') & (df['season_x'] == '2017-18'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Raheem Sterling') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Charlie Taylor') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Kyle Taylor') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Stuart Taylor') & (df['season_x'] == '2017-18'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Antonio Valencia') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Enner Valencia') & (df['season_x'] == '2017-18'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Danny Ward') & (df['season_x'] == '2017-18'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Joel Ward') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Stephen Ward') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Ashley Williams') & (df['season_x'] == '2017-18'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Danny Williams') & (df['season_x'] == '2017-18'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Callum Wilson') & (df['season_x'] == '2017-18'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Kane Wilson') & (df['season_x'] == '2017-18'), 'DEF', df['position'])

### 2018-19

In [ ]:
# Adding Web Names (df uses full names while positions uses web names)

players_raw_2018_19 = pd.read_csv('drive/MyDrive/Colab Notebooks/FYP/Players Raw Data/players_raw_2018-19.csv')

players_raw_2018_19['name'] = players_raw_2018_19[['first_name', 'second_name']].agg(' '.join, axis=1)

players_raw_2018_19 = players_raw_2018_19.drop_duplicates(subset=['web_name', 'name'], keep='first')

players_raw_2018_19['name'] = players_raw_2018_19['name'].apply(unidecode)

df = df.merge(players_raw_2018_19[['name', 'web_name']], how='left', on='name')

df['web_name'] = df['web_name'].apply(lambda x: unidecode(x) if isinstance(x, str) else x)

# Adding Positions

positions_2018_19 = pd.read_csv('drive/MyDrive/Colab Notebooks/FYP/Position Data/2018-19.csv')

# Removing duplicate web names since not doing so would result in the same player having more rows of data

positions_2018_19_unique = positions_2018_19.drop_duplicates(subset='web_name', keep='first')

df = df.merge(positions_2018_19_unique[['web_name', 'position']], how='left', on='web_name')

df['position_x'] = df['position_x'].fillna(df['position_y'])

df = df.drop('position_y', axis=1)
df = df.drop('web_name', axis=1)
df.rename(columns={'position_x': 'position'}, inplace=True)

In [ ]:
# Adding Positions of Players with the same Web Name:

df['position'] = np.where((df['name'] == 'Juninho Bacuna') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Leandro Bacuna') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Ashley Barnes') & (df['season_x'] == '2018-19'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Harvey Barnes') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Joe Bennett') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Ryan Bennett') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Joel Coleman') & (df['season_x'] == '2018-19'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Seamus Coleman') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Tome Davies') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Ben Davies') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Heurelho Gomes') & (df['season_x'] == '2018-19'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Angel Gomes') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Karlan Grant') & (df['season_x'] == '2018-19'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Lee Grant') & (df['season_x'] == '2018-19'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Andre Gray') & (df['season_x'] == '2018-19'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Demarai Gray') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Ben Johnson') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Tyreke Johnson') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Alfie Jones') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Curtis Jones') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Phil Jones') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Kevin Long') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Shane Long') & (df['season_x'] == '2018-19'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Alex McCarthy') & (df['season_x'] == '2018-19'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'James McCarthy') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Nampalys Mendy') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Benjamin Mendy') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Brian Murphy') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Jacob Murphy') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Andreas Pereira') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Ricardo Domingos Barbosa Pereira') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Bobby Reid') & (df['season_x'] == '2018-19'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Winston Reid') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Diego Rico') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Sergio Rico') & (df['season_x'] == '2018-19'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Danny Simpson') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Jack Simpson') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Dale Stephens') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Jack Stephens') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Kazaiah Sterling') & (df['season_x'] == '2018-19'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Raheem Sterling') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Alexis Sanchez') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Davinson Sanchez') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Charlie Taylor') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Kyle Taylor') & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Danny Ward') & (df['element'] == 105) & (df['season_x'] == '2018-19'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Danny Ward') & (df['element'] == 457) & (df['season_x'] == '2018-19'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Joel Ward') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Stephen Ward') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Ashley Williams') & (df['season_x'] == '2018-19'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Danny Williams') & (df['season_x'] == '2018-19'), 'MID', df['position'])

### 2019-20

In [ ]:
# Adding Web Names (df uses full names while positions uses web names)

players_raw_2019_20 = pd.read_csv('drive/MyDrive/Colab Notebooks/FYP/Players Raw Data/players_raw_2019-20.csv')

players_raw_2019_20['name'] = players_raw_2019_20[['first_name', 'second_name']].agg(' '.join, axis=1)

players_raw_2019_20 = players_raw_2019_20.drop_duplicates(subset=['web_name', 'name'], keep='first')

players_raw_2019_20['name'] = players_raw_2019_20['name'].apply(unidecode)

df = df.merge(players_raw_2019_20[['name', 'web_name']], how='left', on='name')

df['web_name'] = df['web_name'].apply(lambda x: unidecode(x) if isinstance(x, str) else x)

# Adding Positions

positions_2019_20 = pd.read_csv('drive/MyDrive/Colab Notebooks/FYP/Position Data/2019-20.csv')

# Removing duplicate web names since not doing so would result in the same player having more rows of data

positions_2019_20_unique = positions_2019_20.drop_duplicates(subset='web_name', keep='first')

df = df.merge(positions_2019_20_unique[['web_name', 'position']], how='left', on='web_name')

df['position_x'] = df['position_x'].fillna(df['position_y'])

df = df.drop('position_y', axis=1)
df = df.drop('web_name', axis=1)
df.rename(columns={'position_x': 'position'}, inplace=True)

In [ ]:
# Adding Positions of Players with the same Web Name:

df['position'] = np.where((df['name'] == 'Maximillian Aarons') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Rolando Aarons') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Brandon Austin') & (df['season_x'] == '2019-20'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Charlie Austin') & (df['season_x'] == '2019-20'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Ashley Barnes') & (df['season_x'] == '2019-20'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Harvey Barnes') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Leon Clarke') & (df['season_x'] == '2019-20'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Matthew Clarke') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Ben Davies') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Tom Davies') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Mark Duffy') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Shane Duffy') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Bruno Miguel Borges Fernandes') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Gedson Carvalho Fernandes') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Kieron Freeman') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Luke Freeman') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Ben Gibson') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Lewis Gibson') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Liam Gibson') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Angel Gomes') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Heurelho da Silva Gomes') & (df['season_x'] == '2019-20'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Anthony Gordon') & (df['season_x'] == '2019-20'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'John-Kymani Gordon') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Andre Gray') & (df['season_x'] == '2019-20'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Demarai Gray') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Isaac Hayden') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Kaine Hayden') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Dean Henderson') & (df['season_x'] == '2019-20'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Jordan Henderson') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Stephen Henderson') & (df['season_x'] == '2019-20'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Daniel James') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Matty James') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Reece James') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Ben Johnson') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Darnell Johnson') & (df['season_x'] == '2019-20'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Tyreke Johnson') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Curtis Jones') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Phil Jones') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Diego Jota') & (df['season_x'] == '2019-20'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Jose Ignacio Peleteiro Romallo') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Lloyd Kelly') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Martin Kelly') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Andy King') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Joshua King') & (df['season_x'] == '2019-20'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'Alfie Lewis') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Harry Lewis') & (df['season_x'] == '2019-20'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Jamal Lewis') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Kevin Long') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Shane Long') & (df['season_x'] == '2019-20'), 'FWD', df['position'])
df['position'] = np.where((df['name'] == 'David Martin') & (df['season_x'] == '2019-20'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Josh Martin') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Alex McCarthy') & (df['season_x'] == '2019-20'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'James McCarthy') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Benjamin Mendy') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Nampalys Mendy') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Andreas Pereira') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Ricardo Domingos Barbosa Pereira') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Dale Stephens') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Jack Stephens') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Alexis Sanchez') & (df['season_x'] == '2019-20'), 'MID', df['position'])
df['position'] = np.where((df['name'] == 'Davinson Sanchez') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Charlie Taylor') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Neil Taylor') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Bobby Thomas') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Jordan Thomas') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Luke Thomas') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Danny Ward') & (df['season_x'] == '2019-20'), 'GKP', df['position'])
df['position'] = np.where((df['name'] == 'Joel Ward') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Brandon Williams') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Neco Williams') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Ashley Young') & (df['season_x'] == '2019-20'), 'DEF', df['position'])
df['position'] = np.where((df['name'] == 'Jack Young') & (df['season_x'] == '2019-20'), 'MID', df['position'])

## Misc Fields

In [ ]:
df['games_played'] = np.where(df['minutes'] > 0, 1, 0)
df['games_played'] = df.groupby(['element', 'season_x'])['games_played'].cumsum()
df['games_played'] = df.groupby(['element', 'season_x'])['games_played'].shift()
df['games_played'] = np.where((df['GW'] == df['GW'].shift()) & (df['element'] == df['element'].shift()), df['games_played'].shift(), df['games_played'])
df['games_played'].fillna(0, inplace=True)

## New Dataset Which Stores Features to be Used in Regressor

In [ ]:
dataset = pd.DataFrame()

dataset['season_x']= df['season_x']
dataset['match_date'] = df['kickoff_time']
dataset['match_date'] = dataset['match_date'].str.slice(0, 10)
dataset['element']= df['element']
dataset['name']= df['name']
dataset['position']= df['position']
dataset['GW']= df['GW']
dataset['opponent_team']= df['opponent_team']
dataset['gw_points']= df['points']
dataset['was_home'] = df['was_home'].astype(int) # changing true to 1 and false to 0
dataset['minutes'] = df['minutes']
dataset

,season_x,match_date,element,name,position,GW,opponent_team,gw_points,was_home,minutes
0,2017-18,2017-08-13,443,Aaron Cresswell,DEF,1,12,0,0,9
1,2017-18,2017-08-19,443,Aaron Cresswell,DEF,2,14,1,0,90
2,2017-18,2017-08-26,443,Aaron Cresswell,DEF,3,13,1,0,90
3,2017-18,2017-09-11,443,Aaron Cresswell,DEF,4,8,6,1,90
4,2017-18,2017-09-16,443,Aaron Cresswell,DEF,5,19,9,0,90
...,...,...,...,...,...,...,...,...,...,...
172854,2023-24,2024-04-20,594,Zeki Amdouni,FWD,34,17,1,0,5
172855,2023-24,2024-04-27,594,Zeki Amdouni,FWD,35,14,7,0,33
172856,2023-24,2024-05-04,594,Zeki Amdouni,FWD,36,15,1,1,5
172857,2023-24,2024-05-11,594,Zeki Amdouni,FWD,37,18,1,0,2


### Calculating Current Season Features

In [ ]:
def calc_stat_per_game(stat):
  df['cumulative_' + stat] = np.where((df['GW'] == df['GW'].shift()) & (df['element'] == df['element'].shift()), (df.groupby(['element', 'season_x'])[stat].cumsum() - df[stat] - df[stat].shift()), df.groupby(['element', 'season_x'])[stat].cumsum() - df[stat])

  dataset[stat + '_per_game'] = (df['cumulative_' + stat] / df['games_played'])
  dataset[stat + '_per_game'] = np.where((df['GW'] == df['GW'].shift()) & (df['element'] == df['element'].shift()), dataset[stat + '_per_game'].shift(), dataset[stat + '_per_game'])
  dataset[stat + '_per_game'].fillna(0, inplace=True)

In [ ]:
def last_5(stat):
  #if stat == 'minutes':
  #  for i in range(1,6):
  #    dataset['prev_game_' + stat + '_' + str(i)] = np.where(df['element'] == df['element'].shift(i), df[stat].shift(i), 0)
  #    dataset['prev_game_' + stat + '_' + str(i)] = np.where((df['GW'] == df['GW'].shift()) & (df['element'] == df['element'].shift()), dataset['prev_game_' + stat + '_' + str(i)].shift(), dataset['prev_game_' + stat + '_' + str(i)])
  #    dataset['prev_game_' + stat + '_' + str(i)] = dataset['prev_game_' + stat + '_' + str(i)].fillna(0)

  #else:
  for i in range(1,6):
    df['prev_game_' + stat + '_' + str(i)] = np.where(df['element'] == df['element'].shift(i), df[stat].shift(i), 0)
    df['prev_game_' + stat + '_' + str(i)] = np.where((df['GW'] == df['GW'].shift()) & (df['element'] == df['element'].shift()), df['prev_game_' + stat + '_' + str(i)].shift(), df['prev_game_' + stat + '_' + str(i)])
    df['prev_game_' + stat + '_' + str(i)] = df['prev_game_' + stat + '_' + str(i)].fillna(0)

In [ ]:
def calc_form(row, stat):
  if row['GW'] == 1:
    return 0
  elif row['GW'] == 2:
    return row[f'prev_game_{stat}_1']
  elif row['GW'] == 3:
    return (row[f'prev_game_{stat}_1'] + row[f'prev_game_{stat}_2']) / 2
  elif row['GW'] == 4:
    return (row[f'prev_game_{stat}_1'] + row[f'prev_game_{stat}_2'] + row[f'prev_game_{stat}_3']) / 3
  elif row['GW'] == 5:
    return (row[f'prev_game_{stat}_1'] + row[f'prev_game_{stat}_2'] + row[f'prev_game_{stat}_3'] + row[f'prev_game_{stat}_4']) / 4
  elif row['GW'] > 5:
    return (row[f'prev_game_{stat}_1'] + row[f'prev_game_{stat}_2'] + row[f'prev_game_{stat}_3'] + row[f'prev_game_{stat}_4'] + row[f'prev_game_{stat}_5']) / 5

#### Minutes Played Per Gameweek

In [ ]:
df['cumulative_minutes'] = np.where((df['GW'] == df['GW'].shift()) & (df['element'] == df['element'].shift()), (df.groupby(['element', 'season_x'])['minutes'].cumsum() - df['minutes'] - df['minutes'].shift()), df.groupby(['element', 'season_x'])['minutes'].cumsum() - df['minutes'])

dataset['minutes_per_gameweek'] = (df['cumulative_minutes'] / (df['player_game_x'] - 1))
dataset['minutes_per_gameweek'] = np.where((dataset['GW'] == dataset['GW'].shift()) & (dataset['element'] == dataset['element'].shift()), dataset['minutes_per_gameweek'].shift(), dataset['minutes_per_gameweek'])
dataset['minutes_per_gameweek'].fillna(0, inplace=True)

#### Minutes Form

In [ ]:
last_5('minutes')
dataset['minutes_form'] = df.apply(lambda row: calc_form(row, stat='minutes'), axis=1)

#### Points per Game

In [ ]:
calc_stat_per_game('points')

#### Form

In [ ]:
last_5('points')
dataset['form'] = df.apply(lambda row: calc_form(row, stat='points'), axis=1)

#### Influence per Game

In [ ]:
calc_stat_per_game('influence')

#### Influence Form

In [ ]:
last_5('influence')
dataset['influence_form'] = df.apply(lambda row: calc_form(row, stat='influence'), axis=1)

#### Creativity per Game

In [ ]:
calc_stat_per_game('creativity')

#### Creativity Form

In [ ]:
last_5('creativity')
dataset['creativity_form'] = df.apply(lambda row: calc_form(row, stat='creativity'), axis=1)

#### Threat per Game

In [ ]:
calc_stat_per_game('threat')

#### Threat Form

In [ ]:
last_5('threat')
dataset['threat_form'] = df.apply(lambda row: calc_form(row, stat='threat'), axis=1)

#### ICT Index per Game

In [ ]:
calc_stat_per_game('ict_index')

#### ICT Index Form

In [ ]:
last_5('ict_index')
dataset['ict_form'] = df.apply(lambda row: calc_form(row, stat='ict_index'), axis=1)

#### BPS per Game

In [ ]:
calc_stat_per_game('bps')

#### BPS Form

In [ ]:
last_5('bps')
dataset['bps_form'] = df.apply(lambda row: calc_form(row, stat='bps'), axis=1)

#### Saves per Game

In [ ]:
calc_stat_per_game('saves')

#### Clean Sheets per Game

In [ ]:
calc_stat_per_game('clean_sheets')

#### Yellow Cards per Game

In [ ]:
calc_stat_per_game('yellow_cards')

#### Red Cards per Game

In [ ]:
calc_stat_per_game('red_cards')

### Previous Season Stats

In [ ]:
def remove_stats(element):
  merged_data['minutes_y'] = np.where(merged_data['element'] == element, None, merged_data['minutes_y'])
  merged_data['points'] = np.where(merged_data['element'] == element, None, merged_data['points'])
  merged_data['influence'] = np.where(merged_data['element'] == element, None, merged_data['influence'])
  merged_data['creativity'] = np.where(merged_data['element'] == element, None, merged_data['creativity'])
  merged_data['threat'] = np.where(merged_data['element'] == element, None, merged_data['threat'])
  merged_data['ict_index'] = np.where(merged_data['element'] == element, None, merged_data['ict_index'])
  merged_data['bps'] = np.where(merged_data['element'] == element, None, merged_data['bps'])
  merged_data['saves'] = np.where(merged_data['element'] == element, None, merged_data['saves'])
  merged_data['clean_sheets'] = np.where(merged_data['element'] == element, None, merged_data['clean_sheets'])
  merged_data['yellow_cards'] = np.where(merged_data['element'] == element, None, merged_data['yellow_cards'])
  merged_data['red_cards'] = np.where(merged_data['element'] == element, None, merged_data['red_cards'])
  merged_data['player_game_x'] = np.where(merged_data['element'] == element, None, merged_data['player_game_x'])
  merged_data['games_played'] = np.where(merged_data['element'] == element, None, merged_data['games_played'])

In [ ]:
def calc_stat_last_season(stat):
  merged_data[stat + '_' + 'per_game'] = np.where(merged_data['games_played'] != 0, (merged_data[stat] / merged_data['games_played']), 0)

  values = merged_data[stat + '_' + 'per_game'].values

  dataset.loc[dataset['season_x'] == next_season, 'previous_season_' + stat + '_per_game'] = values

In [ ]:
seasons = ['2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23']

In [ ]:
for season in seasons:
  with open('/content/drive/MyDrive/Colab Notebooks/FYP/Players Raw Data/players_raw_{}.csv'.format(season), 'rb') as f:
    result = chardet.detect(f.read())

  eos_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FYP/Players Raw Data/players_raw_{}.csv'.format(season), encoding=result['encoding'])

  with open('/content/drive/MyDrive/Colab Notebooks/FYP/Gameweek Data/merged_gw_{}.csv'.format(season), 'rb') as f:
    result = chardet.detect(f.read())

  merged_previous_season = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FYP/Gameweek Data/merged_gw_{}.csv'.format(season), encoding=result['encoding'])

  # Calculating the number of games the player was at the club for and number of games played in previous season

  merged_previous_season['player_game_x'] = merged_previous_season.groupby('element')['element'].transform('count')

  def count_non_zero_minutes(x):
    return (x > 0).sum()

  counts = merged_previous_season.groupby('element')['minutes'].agg(count_non_zero_minutes).reset_index()
  merged_previous_season = pd.merge(merged_previous_season, counts, on='element', suffixes=('', '_count'))
  merged_previous_season.rename(columns={'minutes_count': 'games_played'}, inplace=True)

  eos_df['name'] = eos_df['first_name'] + ' ' + eos_df['second_name']
  eos_df['name'] = eos_df['name'].apply(unidecode)

  eos_df.rename(columns={'id':'element', 'total_points': 'points'}, inplace=True)

  eos_df = pd.merge(eos_df, merged_previous_season[['element', 'player_game_x', 'games_played']], on='element', how='left')
  eos_df.drop_duplicates(inplace=True)

  year_parts = season.split('-')
  new_year_parts = [str(int(part) + 1) for part in year_parts]
  next_season = '-'.join(new_year_parts)

  filtered_dataset = dataset[dataset['season_x'] == next_season]

  merged_data = pd.merge(filtered_dataset, eos_df[['name', 'minutes', 'points', 'influence', 'creativity', 'threat', 'ict_index', 'bps', 'saves', 'clean_sheets', 'yellow_cards', 'red_cards', 'player_game_x', 'games_played']], on='name', how='left')

  # Removing instances where a player inherits the previous season stats of another player with the same name

  if season == '2017-18':
    remove_stats(105)

  if season == '2019-20':
    remove_stats(653)

  # Removing Duplicate Rows caused by 2 players having the same name

  if season == '2018-19':
    mask = (merged_data['element'] == 170) & (merged_data['points'] == 22)
    merged_data = merged_data[~mask]

  if season == '2020-21':
    mask = (merged_data['element'] == 248) & (merged_data['points'] == 52)
    merged_data = merged_data[~mask]
    mask = (merged_data['element'] == 364) & (merged_data['points'] == 0)
    merged_data = merged_data[~mask]

  if season == '2021-22':
    mask = (merged_data['element'] == 432) & (merged_data['points'] == 0)
    merged_data = merged_data[~mask]
    mask = (merged_data['element'] == 499) & (merged_data['points'] == 105)
    merged_data = merged_data[~mask]

  if season == '2022-23':
    mask = (merged_data['element'] == 495) & (merged_data['points'] == 0)
    merged_data = merged_data[~mask]

  merged_data.replace([np.inf, -np.inf], 0, inplace=True)

  # Minutes per Gameweek

  merged_data['minutes_per_gameweek'] = np.where(merged_data['minutes_y'].notna(), (merged_data['minutes_y'] / merged_data['player_game_x']), None)

  prev_minutes_per_gameweek = merged_data['minutes_per_gameweek'].values

  dataset.loc[dataset['season_x'] == next_season, 'previous_season_minutes_per_gameweek'] = prev_minutes_per_gameweek

  # Points per Game

  calc_stat_last_season('points')

  # Influence per Game

  calc_stat_last_season('influence')

  # Creativity per Game

  calc_stat_last_season('creativity')

  # Threat per Game

  calc_stat_last_season('threat')

  # ICT per Game

  calc_stat_last_season('ict_index')

  # BPS per Game

  calc_stat_last_season('bps')

  # Saves per Game

  calc_stat_last_season('saves')

  # Clean Sheets per Game

  calc_stat_last_season('clean_sheets')

  # Yellow Cards per Game

  calc_stat_last_season('yellow_cards')

  # Red Cards per Game

  calc_stat_last_season('red_cards')

#### Fill NA values with mean of Position

In [ ]:
stats = ['previous_season_minutes_per_gameweek', 'previous_season_points_per_game', 'previous_season_influence_per_game', 'previous_season_creativity_per_game', 'previous_season_threat_per_game', 'previous_season_ict_index_per_game', 'previous_season_bps_per_game', 'previous_season_saves_per_game', 'previous_season_clean_sheets_per_game', 'previous_season_yellow_cards_per_game', 'previous_season_red_cards_per_game']

In [ ]:
for stat in stats:
  dataset.loc[dataset['position'] == 'GKP', stat] = dataset.loc[dataset['position'] == 'GKP', stat].fillna(dataset[dataset['position'] == 'GKP'][stat].mean())
  dataset.loc[dataset['position'] == 'DEF', stat] = dataset.loc[dataset['position'] == 'DEF', stat].fillna(dataset[dataset['position'] == 'DEF'][stat].mean())
  dataset.loc[dataset['position'] == 'MID', stat] = dataset.loc[dataset['position'] == 'MID', stat].fillna(dataset[dataset['position'] == 'MID'][stat].mean())
  dataset.loc[dataset['position'] == 'FWD', stat] = dataset.loc[dataset['position'] == 'FWD', stat].fillna(dataset[dataset['position'] == 'FWD'][stat].mean())

### Importing Fixture Data

In [ ]:
fixture_data_2017_18 = []
fixture_data_2018_19 = []
fixture_data_2019_20 = []
fixture_data_2020_21 = []
fixture_data_2021_22 = []
fixture_data_2022_23 = []
fixture_data_2023_24 = []

# 2017-18

path = '/content/drive/MyDrive/Colab Notebooks/FYP/Fixture Data/2017-18/*.csv'
csv_files = glob.glob(path)
csv_files = natsorted(csv_files)

for csv_file in csv_files:
  fixture_df = pd.read_csv(csv_file)
  fixture_data_2017_18.append(fixture_df)

#2018-19

path = '/content/drive/MyDrive/Colab Notebooks/FYP/Fixture Data/2018-19/*.csv'
csv_files = glob.glob(path)
csv_files = natsorted(csv_files)

for csv_file in csv_files:
  fixture_df = pd.read_csv(csv_file)
  fixture_data_2018_19.append(fixture_df)

# 2019-20

path = '/content/drive/MyDrive/Colab Notebooks/FYP/Fixture Data/2019-20/*.csv'
csv_files = glob.glob(path)
csv_files = natsorted(csv_files)

for csv_file in csv_files:
  fixture_df = pd.read_csv(csv_file)
  fixture_data_2019_20.append(fixture_df)

# 2020-21

path = '/content/drive/MyDrive/Colab Notebooks/FYP/Fixture Data/2020-21/*.csv'
csv_files = glob.glob(path)
csv_files = natsorted(csv_files)

for csv_file in csv_files:
  fixture_df = pd.read_csv(csv_file)
  fixture_data_2020_21.append(fixture_df)

# 2021-22

path = '/content/drive/MyDrive/Colab Notebooks/FYP/Fixture Data/2021-22/*.csv'
csv_files = glob.glob(path)
csv_files = natsorted(csv_files)

for csv_file in csv_files:
  fixture_df = pd.read_csv(csv_file)
  fixture_data_2021_22.append(fixture_df)

# 2022-23

path = '/content/drive/MyDrive/Colab Notebooks/FYP/Fixture Data/2022-23/*.csv'
csv_files = glob.glob(path)
csv_files = natsorted(csv_files)

for csv_file in csv_files:
  fixture_df = pd.read_csv(csv_file)
  fixture_data_2022_23.append(fixture_df)

# 2023-24 - Web Scarped from FBref

# Getting urls for all teams

url = 'https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats'
data = requests.get(url)
soup = BeautifulSoup(data.text)

standing_table = soup.select('table.stats_table')[0]

links = standing_table.find_all('a')
links = [l.get('href') for l in links]
links = [l for l in links if '/squads' in l]
team_urls = [f'https://fbref.com{l}' for l in links]

for i, url in enumerate(team_urls):
  parts = url.split('/')
  parts.insert(-1, '2023-2024/matchlogs/c9/schedule')
  url = '/'.join(parts)
  team_urls[i] = url.replace('Stats', 'Scores-and-Fixtures-Premier-League')

team_urls = sorted(team_urls, key=lambda url: url.split('/')[-1])

# Getting data from urls

for i, url in enumerate(team_urls):
  time.sleep(2)
  data = requests.get(url)
  table = pd.read_html(data.text, match = 'Scores & Fixtures')
  fixture_data_2023_24.extend(table)

for i, fixture_df in enumerate(fixture_data_2023_24):
  fixture_df = fixture_df.drop(columns = ['Time', 'Round', 'Day', 'Result', 'Opponent', 'Poss', 'Attendance', 'Captain', 'Referee', 'Formation', 'Match Report', 'Notes'])
  fixture_df.rename(columns = {'Date':'match_date'}, inplace = True)
  fixture_data_2023_24[i] = fixture_df.sort_values('match_date')

In [ ]:
# 2023-24 - Web Scarped from FBref

fixture_data_2023_24 = []

# Getting urls for all teams

url = 'https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats'
data = requests.get(url)
soup = BeautifulSoup(data.text)

standing_table = soup.select('table.stats_table')[0]

links = standing_table.find_all('a')
links = [l.get('href') for l in links]
links = [l for l in links if '/squads' in l]
team_urls = [f'https://fbref.com{l}' for l in links]

for i, url in enumerate(team_urls):
  parts = url.split('/')
  parts.insert(-1, '2023-2024/matchlogs/c9/schedule')
  url = '/'.join(parts)
  team_urls[i] = url.replace('Stats', 'Scores-and-Fixtures-Premier-League')

team_urls = sorted(team_urls, key=lambda url: url.split('/')[-1])

# Getting data from urls

for i, url in enumerate(team_urls):
  time.sleep(2)
  data = requests.get(url)
  table = pd.read_html(data.text, match = 'Scores & Fixtures')
  fixture_data_2023_24.extend(table)

for i, fixture_df in enumerate(fixture_data_2023_24):
  fixture_df = fixture_df.drop(columns = ['Time', 'Round', 'Day', 'Result', 'Opponent', 'Poss', 'Attendance', 'Captain', 'Referee', 'Formation', 'Match Report', 'Notes'])
  fixture_df.rename(columns = {'Date':'match_date'}, inplace = True)
  fixture_data_2023_24[i] = fixture_df.sort_values('match_date')

#### Function for Calculating Team Stats

In [ ]:
def calculate_stats_inner(filtered_df, stat):
  if len(filtered_df) == 0:
    return 0

  else:
    if stat == 'GF':
      return filtered_df['GF'].sum() / len(filtered_df)
    elif stat == 'xG':
      return filtered_df['xG'].sum() / len(filtered_df)
    elif stat == 'GA':
      return filtered_df['GA'].sum() / len(filtered_df)
    elif stat == 'xGA':
      return filtered_df['xGA'].sum() / len(filtered_df)
    elif stat == 'GF_form':
      return filtered_df.tail(5)['GF'].sum() / len(filtered_df.tail(5))
    elif stat == 'GA_form':
      return filtered_df.tail(5)['GA'].sum() / len(filtered_df.tail(5))
    elif stat == 'goalless':
      return filtered_df['GF'].value_counts().get(0, 0) / len(filtered_df)
    elif stat == 'clean_sheets':
      return filtered_df['GA'].value_counts().get(0, 0) / len(filtered_df)

In [ ]:
def calculate_stats(row, stat):
  season_x = row['season_x']
  opponent_team = row['opponent_team']
  match_date = row['match_date']

  if season_x == '2017-18':
    filtered_df = fixture_data_2017_18[opponent_team-1][fixture_data_2017_18[opponent_team-1]['match_date'] < match_date]

  elif season_x == '2018-19':
    filtered_df = fixture_data_2018_19[opponent_team-1][fixture_data_2018_19[opponent_team-1]['match_date'] < match_date]

  elif season_x == '2019-20':
    filtered_df = fixture_data_2019_20[opponent_team-1][fixture_data_2019_20[opponent_team-1]['match_date'] < match_date]

  elif season_x == '2020-21':
    filtered_df = fixture_data_2020_21[opponent_team-1][fixture_data_2020_21[opponent_team-1]['match_date'] < match_date]

  elif season_x == '2021-22':
    filtered_df = fixture_data_2021_22[opponent_team-1][fixture_data_2021_22[opponent_team-1]['match_date'] < match_date]

  elif season_x == '2022-23':
    filtered_df = fixture_data_2022_23[opponent_team-1][fixture_data_2022_23[opponent_team-1]['match_date'] < match_date]

  elif season_x == '2023-24':
    filtered_df = fixture_data_2023_24[opponent_team-1][fixture_data_2023_24[opponent_team-1]['match_date'] < match_date]

  filtered_df = filtered_df.dropna()
  return calculate_stats_inner(filtered_df, stat)

#### Opposition Team Goals Scored per Game

In [ ]:
dataset['opp_team_goals_scored_per_game'] = dataset.apply(calculate_stats, args=('GF',), axis=1)

#### Opposition Team Expected Goals Scored per Game

In [ ]:
dataset['opp_team_xG_per_game'] = dataset.apply(calculate_stats, args=('xG',), axis=1)

#### Opposition Team Goals Conceded per Game

In [ ]:
dataset['opp_team_goals_conceded_per_game'] = dataset.apply(calculate_stats, args=('GA',), axis=1)

#### Opposition Team Expected Goals Conceded per Game

In [ ]:
dataset['opp_team_xGA_per_game'] = dataset.apply(calculate_stats, args=('xGA',), axis=1)

#### Opposition Team Goals Scored Form

In [ ]:
dataset['opp_team_goals_scored_form'] = dataset.apply(calculate_stats, args=('GF_form',), axis=1)

#### Opposition Team Goals Conceded Form

In [ ]:
dataset['opp_team_goals_conceded_form'] = dataset.apply(calculate_stats, args=('GA_form',), axis=1)

#### Opposition Team Goalless Games per Game

In [ ]:
dataset['opp_team_goalless_games_per_game'] = dataset.apply(calculate_stats, args=('goalless',), axis=1)

#### Opposition Team Clean Sheets per Game

In [ ]:
dataset['opp_team_clean_sheets_per_game'] = dataset.apply(calculate_stats, args=('clean_sheets',), axis=1)

### Replacing inf with 0

In [ ]:
dataset.replace([np.inf, -np.inf], 0, inplace=True)

## Saving Next Gameweek Dataset as CSV

In [ ]:
next_gameweek_dataset = dataset[(dataset['GW'] == min_gw) & (dataset['season_x'] == '2023-24')] # selecting next gameweek data

next_gameweek_dataset.drop(['match_date', 'opponent_team', 'gw_points', 'minutes'], axis=1).to_csv('next_gameweek_dataset.csv', index=False)
files.download('next_gameweek_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Saving Training and Testing Dataset as CSV

In [ ]:
dataset = dataset[(dataset['GW'] != min_gw) | (dataset['season_x'] != '2023-24')] # removing next gameweek data

dataset.drop(['match_date', 'opponent_team'], axis=1).to_csv('train_test_dataset.csv', index=False)
files.download('train_test_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>